In [1]:
!pip install keras-tcn
!pip install -U finance-datareader

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN

/home/work/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2023-02-06 06:09:15.059650: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 06:09:15.310299: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-06 06:09:19.851631: W tensorflow/compiler/xla/stream_executor/platform/default/

In [3]:
kospi_train = fdr.DataReader('KS11', '1990-01-01','2017-12-31')
#kospi_train['Change1'] = kospi_train['Close']/kospi_train['Close'].shift(1) - 1
kospi_train['Change7'] = kospi_train['Close']/kospi_train['Close'].shift(7) - 1
#kospi_train['Change15'] = kospi_train['Close']/kospi_train['Close'].shift(15) - 1
#kospi_train['Change30'] = kospi_train['Close']/kospi_train['Close'].shift(30) - 1
print(kospi_train.head())
print(kospi_train.columns)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
1996-12-11  705.989990  709.479980  704.429993  704.679993  704.679993   
1996-12-12  705.109985  706.010010  688.739990  689.380005  689.380005   
1996-12-13  690.440002  695.719971  677.640015  689.070007  689.070007   
1996-12-16  686.969971  686.969971  667.710022  673.919983  673.919983   
1996-12-17  675.349976  680.090027  660.390015  663.349976  663.349976   

             Volume  Change7  
Date                          
1996-12-11  28000.0      NaN  
1996-12-12  25900.0      NaN  
1996-12-13  26500.0      NaN  
1996-12-16  22800.0      NaN  
1996-12-17  31600.0      NaN  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change7'], dtype='object')


In [4]:
kospi_train = kospi_train.dropna()
print(kospi_train.head())
print(kospi_train.columns)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
1996-12-20  695.330017  703.510010  692.469971  700.869995  700.869995   
1996-12-23  691.979980  697.119995  684.809998  684.809998  684.809998   
1996-12-24  683.109985  687.059998  677.559998  677.760010  677.760010   
1996-12-26  676.210022  676.210022  655.239990  659.010010  659.010010   
1996-12-27  654.820007  657.039978  643.570007  651.219971  651.219971   

             Volume   Change7  
Date                           
1996-12-20  30000.0 -0.005407  
1996-12-23  21500.0 -0.006629  
1996-12-24  23800.0 -0.016413  
1996-12-26  25500.0 -0.006542  
1996-12-27  35100.0 -0.015377  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change7'], dtype='object')


In [5]:
kospi_train_close = kospi_train['Close']
#kospi_train_c1 = kospi_train['Change1']
kospi_train_c7 = kospi_train['Change7']
#kospi_train_c15 = kospi_train['Change15']
#kospi_train_c30 = kospi_train['Change30']
print(kospi_train_close, kospi_train_c7)

Date
1996-12-20     700.869995
1996-12-23     684.809998
1996-12-24     677.760010
1996-12-26     659.010010
1996-12-27     651.219971
                 ...     
2017-12-21    2429.830078
2017-12-22    2440.540039
2017-12-26    2427.340088
2017-12-27    2436.669922
2017-12-28    2467.489990
Name: Close, Length: 5039, dtype: float64 Date
1996-12-20   -0.005407
1996-12-23   -0.006629
1996-12-24   -0.016413
1996-12-26   -0.006542
1996-12-27   -0.015377
                ...   
2017-12-21   -0.016856
2017-12-22   -0.008314
2017-12-26   -0.021451
2017-12-27   -0.013286
2017-12-28   -0.005874
Name: Change7, Length: 5039, dtype: float64


In [6]:
kospi_test = fdr.DataReader('KS11', '2018-01-01')
#kospi_test['Change1'] = kospi_test['Close']/kospi_test['Close'].shift(1) - 1
kospi_test['Change7'] = kospi_test['Close']/kospi_test['Close'].shift(7) - 1
#kospi_test['Change15'] = kospi_test['Close']/kospi_test['Close'].shift(15) - 1
#kospi_test['Change30'] = kospi_test['Close']/kospi_test['Close'].shift(30) - 1
print(kospi_test.head())
print(kospi_test.columns)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2018-01-02  2474.860107  2481.020020  2465.939941  2479.649902  2479.649902   
2018-01-03  2484.629883  2493.399902  2481.909912  2486.350098  2486.350098   
2018-01-04  2502.500000  2502.500000  2466.449951  2466.459961  2466.459961   
2018-01-05  2476.850098  2497.520020  2475.510010  2497.520020  2497.520020   
2018-01-08  2510.699951  2515.370117  2494.179932  2513.280029  2513.280029   

            Volume  Change7  
Date                         
2018-01-02  262200      NaN  
2018-01-03  331100      NaN  
2018-01-04  333800      NaN  
2018-01-05  308800      NaN  
2018-01-08  311400      NaN  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change7'], dtype='object')


In [7]:
kospi_test = kospi_test.dropna()
print(kospi_test.head())
print(kospi_test.columns)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2018-01-11  2496.860107  2502.520020  2483.820068  2487.909912  2487.909912   
2018-01-12  2499.870117  2500.870117  2482.149902  2496.419922  2496.419922   
2018-01-15  2510.969971  2513.300049  2496.179932  2503.729980  2503.729980   
2018-01-16  2504.159912  2524.550049  2498.550049  2521.739990  2521.739990   
2018-01-17  2517.189941  2521.969971  2506.489990  2515.429932  2515.429932   

            Volume   Change7  
Date                          
2018-01-11  444300  0.003331  
2018-01-12  438800  0.004050  
2018-01-15  404600  0.015111  
2018-01-16  341600  0.009698  
2018-01-17  336700  0.000855  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change7'], dtype='object')


In [8]:
kospi_test_close = kospi_test['Close']
#kospi_test_c1 = kospi_test['Change1']
kospi_test_c7 = kospi_test['Change7']
#kospi_test_c15 = kospi_test['Change15']
#kospi_test_c30 = kospi_test['Change30']
print(kospi_test_close, kospi_test_c7)

Date
2018-01-11    2487.909912
2018-01-12    2496.419922
2018-01-15    2503.729980
2018-01-16    2521.739990
2018-01-17    2515.429932
                 ...     
2023-01-31    2425.080078
2023-02-01    2449.800049
2023-02-02    2468.879883
2023-02-03    2480.399902
2023-02-06    2449.310059
Name: Close, Length: 1247, dtype: float64 Date
2018-01-11    0.003331
2018-01-12    0.004050
2018-01-15    0.015111
2018-01-16    0.009698
2018-01-17    0.000855
                ...   
2023-01-31    0.023966
2023-02-01    0.029181
2023-02-02    0.030736
2023-02-03    0.021342
2023-02-06   -0.007834
Name: Change7, Length: 1247, dtype: float64


In [9]:
kospi_train_close = kospi_train_close.values
#kospi_train_c1 = 100 * kospi_train_c1.values
kospi_train_c7 = 100 * kospi_train_c7.values
#kospi_train_c15 = 100 * kospi_train_c15.values
#kospi_train_c30 = 100 * kospi_train_c30.values

kospi_test_close = kospi_test_close.values
#kospi_test_c1 = 100 * kospi_test_c1.values
kospi_test_c7 = 100 * kospi_test_c7.values
#kospi_test_c15 = 100 * kospi_test_c15.values
#kospi_test_c30 = 100 * kospi_test_c30.values
print(kospi_test_c7)

[ 0.33311194  0.40500427  1.51107334 ...  3.07356499  2.13417083
 -0.78341781]


In [10]:
lookback_window = 60
x_train_7, y_train_7 = [], []

In [11]:
for i in range(lookback_window, len(kospi_train_close)-6):
    x_train_7.append(kospi_train_close[i - lookback_window:i])
    y_train_7.append(kospi_train_c7[i+6])

In [12]:
for index in range(len(y_train_7)):
    if y_train_7[index]>= -0.1 and y_train_7[index] < 0.1:
        y_train_7[index] = 'neutral'
    elif y_train_7[index] >= 0.1:
        y_train_7[index] = 'increase'
    else:
        y_train_7[index] = 'decrease'

In [14]:
x_train_7 = np.array(x_train_7)
y_train_7 = np.array(y_train_7)
x_train_7 = x_train_7.reshape((4973, -1, 1))
#y_train_7 = y_train_7.reshape((-1, 1))

print(x_train_7.shape)
print(y_train_7.shape)

(4973, 60, 1)
(4973,)


In [15]:
x_test_7, y_test_7 = [], []

In [16]:
for i in range(lookback_window, len(kospi_test_close)-6):
    x_test_7.append(kospi_test_close[i - lookback_window:i])
    y_test_7.append(kospi_test_c7[i+6])

In [17]:
for index in range(len(y_test_7)):
    if y_test_7[index]>= -0.1 and y_test_7[index] < 0.1:
        y_test_7[index] = 'neutral'
    elif y_test_7[index] >= 0.1:
        y_test_7[index] = 'increase'
    else:
        y_test_7[index] = 'decrease'

In [20]:
x_test_7 = np.array(x_test_7)
y_test_7 = np.array(y_test_7)
x_test_7 = x_test_7.reshape((1181, -1, 1))
#y_test_7 = y_test_7.reshape((-1, 1))

In [21]:
print(x_test_7.shape)
print(y_test_7.shape)

(1181, 60, 1)
(1181,)


In [22]:
print(y_test_7)

['increase' 'increase' 'increase' ... 'increase' 'increase' 'decrease']


In [24]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
encoder = LabelEncoder()
encoder.fit(y_train_7)

y_train_7_e = to_categorical(encoder.transform(y_train_7))
y_test_7_e = to_categorical(encoder.transform(y_test_7))

In [25]:
print(y_test_7_e.shape)

(1181, 3)


In [26]:
model_1 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        dilations=[1, 2, 4, 8, 16]
        ),
    Dense(3, activation='softmax')
])

model_1.summary()
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_2 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=3,
        dilations=[1, 2, 4, 8, 16]
        ),
    Dense(3, activation='softmax')
])

model_2.summary()
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_3 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=4,
        dilations=[1, 2, 4, 8, 16]
        ),
    Dense(3, activation='softmax')
])

model_3.summary()
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_4 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=5,
        dilations=[1, 2, 4, 8, 16]
        ),
    Dense(3, activation='softmax')
])

model_4.summary()
model_4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_5 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=6,
        dilations=[1, 2, 4, 8, 16]
        ),
    Dense(3, activation='softmax')
])

model_5.summary()
model_5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-02-06 06:13:45.266064: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 06:13:45.964844: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10315 MB memory:  -> device: 0, name: CUDA GPU, pci bus id: 0000:d5:00.0, compute capability: 8.6
2023-02-06 06:13:46.058458: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 10.07G (10816061440 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-02-06 06:13:46.081943: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 9.07G (9734455296 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn (TCN)                   (None, 64)                74624     
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 74,819
Trainable params: 74,819
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_1 (TCN)                 (None, 64)                111552    
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 111,747
Trainabl

In [27]:
print('Train model_k2...')
model_1.fit(x_train_7, y_train_7_e, epochs=100, verbose=1)
p_test_1 = model_1.predict(x_test_7)

print('Train model_k3...')
model_2.fit(x_train_7, y_train_7_e, epochs=100, verbose=1)
p_test_2 = model_2.predict(x_test_7)

print('Train model_k4...')
model_3.fit(x_train_7, y_train_7_e, epochs=100, verbose=1)
p_test_3 = model_3.predict(x_test_7)

print('Train model_k5...')
model_4.fit(x_train_7, y_train_7_e, epochs=100, verbose=1)
p_test_4 = model_4.predict(x_test_7)

print('Train model_k6...')
model_5.fit(x_train_7, y_train_7_e, epochs=100, verbose=1)
p_test_5 = model_5.predict(x_test_7)

Train model_k2...
Epoch 1/100


2023-02-06 06:14:04.477316: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8400
2023-02-06 06:14:05.157531: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:219] failed to create cublas handle: cublas error
2023-02-06 06:14:05.157864: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops.cc:1152 : NOT_FOUND: No algorithm worked!  Error messages:
  Profiling failure on CUDNN engine 1#TC: UNKNOWN: CUDNN_STATUS_EXECUTION_FAILED
in tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc(4294): 'cudnnConvolutionForward( cudnn.handle(), alpha, input_nd_.handle(), input_data.opaque(), filter_.handle(), filter_data.opaque(), conv_.handle(), ToConvForwardAlgo(algo), scratch_memory.opaque(), scratch_memory.size(), beta, output_nd_.handle(), output_data.opaque())'
  Profiling failure on CUDNN engine 1: UNKNOWN: CUDNN_STATUS_EXECUTION_FAILED
in tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc(4294): 'cudnnConvolution

NotFoundError: Graph execution error:

Detected at node 'sequential/tcn/residual_block_0/conv1D_0/Conv1D' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib/python3.8/a

In [25]:
model_1.save('/home/work/nlp/TCN_models/classification_model/v6/kospi_1day_classification_v6.h5')
model_7.save('/home/work/nlp/TCN_models/classification_model/v6/kospi_7days_classification_v6.h5')
model_15.save('/home/work/nlp/TCN_models/classification_model/v6/kospi_15days_classification_v6.h5')
model_30.save('/home/work/nlp/TCN_models/classification_model/v6/kospi_30days_classification_v6.h5')

In [27]:
model_1.evaluate(x_test_1, y_test_1_e)[1]
model_7.evaluate(x_test_7, y_test_7_e)[1]
model_15.evaluate(x_test_15, y_test_15_e)[1]
model_30.evaluate(x_test_30, y_test_30_e)[1]

36/36 [==============================] - 1s 25ms/step - loss: 34.5452 - accuracy: 0.4872


0.4871794879436493